<a href="https://colab.research.google.com/github/sonjoy1s/kaggle_project/blob/main/kdd_cyberattack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/kddcup.data_10_percent.zip")
df.head()

,0,tcp,http,SF,181,5450,0.1,0.2,0.3,0.4,...,9.1,1.00.1,0.00.6,0.11,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   0        494020 non-null  int64  
 1   tcp      494020 non-null  object 
 2   http     494020 non-null  object 
 3   SF       494020 non-null  object 
 4   181      494020 non-null  int64  
 5   5450     494020 non-null  int64  
 6   0.1      494020 non-null  int64  
 7   0.2      494020 non-null  int64  
 8   0.3      494020 non-null  int64  
 9   0.4      494020 non-null  int64  
 10  0.5      494020 non-null  int64  
 11  1        494020 non-null  int64  
 12  0.6      494020 non-null  int64  
 13  0.7      494020 non-null  int64  
 14  0.8      494020 non-null  int64  
 15  0.9      494020 non-null  int64  
 16  0.10     494020 non-null  int64  
 17  0.12     494020 non-null  int64  
 18  0.13     494020 non-null  int64  
 19  0.14     494020 non-null  int64  
 20  0.15     494020 non-null  

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
0,494020.0,47.979400,707.747185,0.0,0.00,0.0,0.00,58329.0
181,494020.0,3025.616054,988219.101229,0.0,45.00,520.0,1032.00,693375640.0
5450,494020.0,868.523151,33040.034049,0.0,0.00,0.0,0.00,5155468.0
0.1,494020.0,0.000045,0.006673,0.0,0.00,0.0,0.00,1.0
0.2,494020.0,0.006433,0.134805,0.0,0.00,0.0,0.00,3.0
0.3,494020.0,0.000014,0.005510,0.0,0.00,0.0,0.00,3.0
0.4,494020.0,0.034519,0.782103,0.0,0.00,0.0,0.00,30.0
0.5,494020.0,0.000152,0.015520,0.0,0.00,0.0,0.00,5.0
1,494020.0,0.148245,0.355343,0.0,0.00,0.0,0.00,1.0
0.6,494020.0,0.010212,1.798328,0.0,0.00,0.0,0.00,884.0


In [7]:
df.drop(columns=['0.14','0.15'],inplace=True)
df.head()

,0,tcp,http,SF,181,5450,0.1,0.2,0.3,0.4,...,9.1,1.00.1,0.00.6,0.11,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [8]:
df = df.drop_duplicates()


In [9]:
df.head()

,0,tcp,http,SF,181,5450,0.1,0.2,0.3,0.4,...,9.1,1.00.1,0.00.6,0.11,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [10]:
df.columns

Index(['0', 'tcp', 'http', 'SF', '181', '5450', '0.1', '0.2', '0.3', '0.4',
       '0.5', '1', '0.6', '0.7', '0.8', '0.9', '0.10', '0.12', '0.13', '0.16',
       '8', '8.1', '0.00', '0.00.1', '0.00.2', '0.00.3', '1.00', '0.00.4',
       '0.00.5', '9', '9.1', '1.00.1', '0.00.6', '0.11', '0.00.7', '0.00.8',
       '0.00.9', '0.00.10', '0.00.11', 'normal.'],
      dtype='object')

In [11]:
cols = [
    'duration',                 # 0
    'protocol_type',            # tcp
    'service',                  # http
    'flag',                     # SF
    'src_bytes',                # 181
    'dst_bytes',                # 5450
    'land',                     # 0.1
    'wrong_fragment',           # 0.2
    'urgent',                   # 0.3
    'hot',                      # 0.4
    'num_failed_logins',        # 0.5
    'logged_in',                # 1
    'num_compromised',          # 0.6
    'root_shell',               # 0.7
    'su_attempted',             # 0.8
    'num_root',                 # 0.9
    'num_file_creations',       # 0.10
    'num_shells',               # 0.12
    'num_access_files',         # 0.13
    'is_host_login',            # 0.16
    'count',                    # 8
    'srv_count',                # 8.1
    'serror_rate',              # 0.00
    'srv_serror_rate',          # 0.00.1
    'rerror_rate',              # 0.00.2
    'srv_rerror_rate',          # 0.00.3
    'same_srv_rate',            # 1.00
    'diff_srv_rate',            # 0.00.4
    'srv_diff_host_rate',       # 0.00.5
    'dst_host_count',           # 9
    'dst_host_srv_count',       # 9.1
    'dst_host_same_srv_rate',   # 1.00.1
    'dst_host_diff_srv_rate',   # 0.00.6
    'dst_host_same_src_port_rate', # 0.11
    'dst_host_srv_diff_host_rate',  # 0.00.7
    'dst_host_serror_rate',     # 0.00.8
    'dst_host_srv_serror_rate', # 0.00.9
    'dst_host_rerror_rate',     # 0.00.10
    'dst_host_srv_rerror_rate', # 0.00.11
    'label'                     # normal.
]


In [12]:
df.columns = cols


In [13]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
